<a href="https://colab.research.google.com/github/simon-clematide/casdmit-fs21/blob/master/notebooks/zora_dewey_fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sprachidentifikation mit fasttext
Dieses Notebook demonstriert, wie einfach man ein gutes LID-Modell mit fastText trainieren kann.
Wir arbeiten mit der fasttext Python-Bibliothek.
Aus Effizienzgründen arbeiten wir hier mit einem kleineren Trainingsdatensatz.

# Installation
## Nur notwendig, wenn nicht in colab/binder VM
Conda-Umgebung mit FastText bereitstellen im Terminal und dann Jupyter Notebook nochmals starten.

```bash
conda activate base
conda create --name lid --clone base
conda activate lid
conda install -c conda-forge fasttext 
# danach neues Terminal öffnen und conda aktivieren

# fasttext Python Modul installieren
# pip install fasttext # hat Bug bei test_label()
pip install git+https://github.com/facebookresearch/fastText.git@b64e359d5485dda4b4b5074494155d18e25c8d13  # braucht mehr Zeit fürs Kompilieren

# danach Jupyter neu starten
jupyter notebook
```

## In colab das Python fasttext Package installieren
Aktuellere Version hat [Bug](https://stackoverflow.com/questions/61787119/fasttext-0-9-2-why-is-recall-nan) in der Evaluationsfunktion korrigiert (

In [ ]:
# ! pip install fasttext # schnell zu installieren, aber hat Bug bei test_label()
! pip install git+https://github.com/facebookresearch/fastText.git@b64e359d5485dda4b4b5074494155d18e25c8d13  # braucht mehr Zeit fürs Kompilieren

# Datenset

In [ ]:
! curl https://files.ifi.uzh.ch/cl/siclemat/lehre/fs21/casdmit/impresso-lid/impresso-lid.tsv -o impresso-lid.tsv

### Format des Datensets 
 - Pro tabulator-separierte Zeile gibt es 2 Spalten
 - Spalte 1: Sprachlabel
 - Spalte 2: Textstücke (ca. 400 Buchstaben) 

In [ ]:
! head -n 10 impresso-lid.tsv

### Statistiken zum Datenset

In [ ]:
! wc impresso-lid.tsv

In [ ]:
!  cut -f 1 < impresso-lid.tsv | sort | uniq -c | sort -rn 

## Aufteilen der Daten in Trainings- und Testdaten
Erstellen von Training und Testdaten (Originaldaten sind zufällig geordnet)

In [ ]:
! head -n 900 < impresso-lid.tsv > impresso-lid-train.tsv
! tail -n 100 < impresso-lid.tsv > impresso-lid-test.tsv

In [ ]:
! echo TRAINING DATA STATISTICS
! cut -f 1 < impresso-lid-train.tsv | sort | uniq -c | sort -rn 
! echo TEST DATA STATISTICS
! cut -f 1 < impresso-lid-test.tsv | sort | uniq -c | sort -rn 

# Trainieren von Modell mit Python-Package
 - Dokumentation siehe https://fasttext.cc/docs/en/python-module.html

In [ ]:
import fasttext

In [ ]:
model = fasttext.train_supervised(
    input='impresso-lid-train.tsv', 
    epoch=10,  # Wie oft werden die Trainingsdaten benutzt
    minn=2,    # Minimal Subword-Länge in Buchstaben  
    maxn=2,    # Maximale Subword-Länge in Buchstaben 
    dim=5,     # Dimensionalität der Vektoren für die Repräsentation der Wörter und Subwords
    lr=0.1     # Learning Rate (Lernrate): Wie stark wird ein Fehler bestraft? 
    )

## Inspizieren des gelernten Modells

Welche Labels/Klassen kennt das Modell?

In [ ]:
print(model.labels)

Einen String klassifizieren und die Wahrscheinlichkeitsverteilung über allen möglichen Sprachen erhalten:

In [ ]:
model.predict("Welche Sprache ist das? Luxemburgisch?",  
              k=5  # Gib die 5 besten Klassen aus
              )


Den gelernten Vektor eines Worts anzeigen.

In [ ]:
word_id = model.get_word_id("Sprache")
print(word_id)
model.get_input_vector(word_id)

Ids von Wort und allen seinen Buchstaben-N-Grammen erhalten.

In [ ]:
model.get_subwords('Sprache')

Was passiert hier?

In [ ]:
model.get_subwords('???')

Systematisches Testen des Minimodells auf Testdaten

In [ ]:
model.test("impresso-lid-test.tsv")

In [ ]:
def print_results(N, p, r):
    "Pretty print performance: N=Number of Samples, P/R@1=Precision/Recall of best prediction Acc=Accuracy "
    print(f"N\t{N}")
    print(f"P@1\t{p:.2f}")
    print(f"R@1\t{r:.2f}")
    print(f"Acc\t{r:.2f}")

In [ ]:
print_results(*model.test("impresso-lid-test.tsv"))

Detaillierte Evaluation zu jedem einzelnen Label:
 - Precision: Anteil korrekter Klassifikationen einer Klasse
 - Recall: Anteil korrekt klassifizierter Elemente einer Klasse
 - f1score: Harmonisches Mittel von Precision und Recall

In [ ]:
model.test_label('impresso-lid-test.tsv',k=1)

## Vorhersagen und Wahrheit anzeigen

In [ ]:
test_data = []
with open("impresso-lid-test.tsv", mode="r",encoding="utf-8") as testfile:
    for line in testfile:
        test_data.append(line.strip().split("\t"))
test_data[:3]


In [ ]:
from collections import Counter
confusion_matrix = Counter()
# If given a list of strings, it will return a list of results as usually received for a single line of text.
predictions,probs = model.predict([text for _,text in test_data])
print('Predictions',predictions)
print('Probabilities',probs)
for i,preds in enumerate(predictions):
    confusion_matrix[(test_data[i][0],preds[0])] += 1

for p in confusion_matrix.most_common():
    print(p)

# Verbessern des Modells
Verbessern des Modells: Z.B. mehr Epochen, mehr Dimensionen, längere Buchstaben-N-Gramme, ...

Wichtigste Parameter:
```
   epoch N  # Beim Lernen wird das ganze Trainingsset N mal benutzt. Beeinflusst die Dauer des Trainings linear!
   dim N    # Länge der gelernten Vektoren für Wörter und Buchstaben-N-Gramme
   lr 0.N   # Initiale Lernrate: Bestimmt, wie stark die Vektoren verändert werden, wenn Fehler passieren. Während des Lernens wird die Lernrate immer kleiner.
   mmin N   # Minimale Länge der Subwords, d.h. Buchstaben-N-Gramme
   maxn N   # Maximale Länger der Subwords, d.h. Buchstaben-N-Gramme (falls N=0, werden keine Subwords benutzt, nur Wörter)
```

In [ ]:
model = fasttext.train_supervised(
    input='impresso-lid-train.tsv', 
    epoch=10,  # Wie oft werden die Trainingsdaten benutzt
    minn=2,    # Minimale Subword-Länge in Buchstaben  
    maxn=2,    # Maximale Subword-Länge in Buchstaben 
    dim=5,     # Dimensionalität der Vektoren für die Repräsentation der Wörter und Subwords
    lr=0.1     # Learning Rate (Lernrate): Wie stark wird ein Fehler bestraft? 
    )
print_results(*model.test("impresso-lid-test.tsv"))
model.test_label('impresso-lid-test.tsv',k=1)

# LID Shared-Task
Arbeitet in Zweiergruppen und versucht, ein besseres Modell zu trainieren. Tragt die beste Konfiguration im Spreadsheet https://cutt.ly/casdmit-fs21-lid ein.

 - Reflexion: Welche Hyperparameter scheinen keinen wesentlichen Einfluss zu haben? Welche den grössten Einfluss?